In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [2]:
checkpoint_path = "./cloning/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [58]:
#      CLONE MODEL EAGER ENABLED
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 3

train, test = rsc.get_titanic_dataset()


model = rsc.get_model(None)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])

"""
clone_fn = Clone_Model.clone_fn
clone = tf.keras.models.clone_model(model, input_tensors=model.inputs, clone_function=clone_fn)

z_in = np.ones(shape=(20,4))
assert np.allclose(model.predict(z_in), clone.predict(z_in)), "Wrooong"
"""

clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs = 1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])
                

3/3 [==============================] - 0s 45ms/step - loss: 1.5399 - my_placeholder_metric_dense_1: 2208.6514 - observed_samples_sat_dense_1: 40.7466 - my_placeholder_metric_dense_2: 1242.8102 - observed_samples_sat_dense_2: 40.0302 - my_placeholder_metric_dense_3: 648.5023 - observed_samples_sat_dense_3: 41.6745 - my_placeholder_metric_dense_4: 630.7831 - observed_samples_sat_dense_4: 41.4525 - my_placeholder_metric_predictions: 20.0000 - observed_samples_sat_predictions: 40.8636 - val_loss: 0.9327 - val_my_placeholder_metric_dense_1: 2100.5286 - val_observed_samples_sat_dense_1: 80.7466 - val_my_placeholder_metric_dense_2: 1193.6204 - val_observed_samples_sat_dense_2: 80.0302 - val_my_placeholder_metric_dense_3: 612.4310 - val_observed_samples_sat_dense_3: 81.6745 - val_my_placeholder_metric_dense_4: 591.7574 - val_observed_samples_sat_dense_4: 81.4525 - val_my_placeholder_metric_predictions: 20.0000 - val_observed_samples_sat_predictions: 80.8636


In [33]:
#(None, 21, 5)[1:]*2
np.zeros(shape=(None))

array(0.)